In [1]:

import random
import sys 

# Para claves de largo 200+ el algoritmo tira recursion limit, si lo seteamos un poco mas alto funciona correctamente.
sys.setrecursionlimit(3000)

def miller_rabin(n: int, k: int) -> bool:
    if n == 2:
        return True
    if n % 2 == 0:
        return False
    
    # Transformar el n-1
    d = n-1
    r = 0
    while d % 2 == 0:
        r += 1
        d = d // 2
    # n = (2**r)*d +1 
    
    salir = 0
    for i in range(k):
        a = random.randint(2, n-2)
        actual = exp_mod(a, d, n) # inicial
        if actual == 1 or actual == n-1:
            continue
        # empezamos desde a elevado a d y vamos elevanto hasta llegar a (a ** n-1).
        for _ in range(r-1):
            actual = exp_mod(actual, 2, n)
            if actual == n-1:
                # nos la jugamos por primo
                salir = 1
                break
        if salir == 1:
            salir = 0
            continue
        # Si vimos puros 1 o cualquier cosa distinta a -1 en los r casos entonces llegamos aquí.
        return False
    return True
            

def generar_primo(l: int) -> int:
    for i in range(3000):
        r1 = expo(10, l-1)
        r2 = expo(10, l) -1
        a = random.randint(r1, r2)
        if miller_rabin(a, 100):
            return a


In [2]:

def alg_ext_euclides(a: int, b:int) -> (int, int, int):
    if b > a: raise Exception("El primer numero tiene que ser mayor al segundo" )
    if a <= 0: raise Exception("El primer numero tiene que ser mayor a 0" )
    r = [a, b]
    s = [1, 0]
    t = [0, 1]
    
    while r[1] != 0:
        aux = r[0] // r[1]
        r = [r[1], r[0] - aux * r[1]]
        s = [s[1], s[0] - aux * s[1]]
        t = [t[1], t[0] - aux * t[1]]
        
    return r[0], s[0], t[0]
    

def exp_mod(a:int , b:int, n:int) -> int:
    if a < 0 or b < 0 or n <= 0: raise Exception("Error de argumentos")
    if b == 0:
        return 1
    elif (b % 2) == 0:
        t = exp_mod(a, b//2, n)
        return (t*t) % n
    else:
        t = exp_mod(a, (b-1) // 2, n)
        return (t*t*a) % n

def expo(a:int , b:int) -> int:
    if b == 0:
        return 1
    elif (b % 2) == 0:
        t = expo(a, b//2)
        return (t*t)
    else:
        t = expo(a, (b-1) // 2)
        return (t*t*a)

In [3]:
def inverso(a:int, n:int) -> int:
    if n < 2: raise Exception("n tiene que ser mayor o igual a 2")
    if a < 2: raise Exception("a tiene que ser mayor o igual a 1")
    if a > n:
        mcd, s, t = alg_ext_euclides(a,n)
        if mcd != 1: raise Exception("los inputs tienen que ser primos relativos")
        # Si tenemos que el inverso es negativo hay que pasarlo a su positivo en modulo n
        if s < 0: 
            s = n + s
        return s
    else:
        mcd, s, t = alg_ext_euclides(n,a)
        if t < 0:
            t = n + t
        if mcd != 1: raise Exception("los inputs tienen que ser primos relativos")
        return t

In [4]:
def generar_clave(l:int):
    p = generar_primo(l)
    q = generar_primo(l)
    n = p * q
    phin = (p-1)*(q-1)
    while True:
        d = random.randint(1, phin)
        mcd, _, _ = alg_ext_euclides(phin, d)
        if mcd == 1:
            break
    e = inverso(d, phin)
    with open("private_key.txt", "w") as file:
        file.write(f"{d}\n")
        file.write(f"{n}\n")
    with open("public_key.txt", "w") as file:
        file.write(f"{e}\n")
        file.write(f"{n}\n")

def enc(m: int) -> int:
    with open("public_key.txt", "r") as file:
        lines = file.readlines()
        e = int(lines[0].strip())
        n = int(lines[1].strip())
    if m >= n: raise Exception("el mensaje es más largo que n")
    return exp_mod(m, e, n)

def dec(m: int) -> int:
    with open("private_key.txt", "r") as file:
        lines = file.readlines()
        d = int(lines[0].strip())
        n = int(lines[1].strip())
    if m >= n: raise Exception("el mensaje es más largo que n")
    return exp_mod(m, d, n)

Descomentar las siguientes lineas para probarlo, ojo que con 200 por consola me tiraba recursion error pero en jupyter me funcionaba bien... con 150 funciona en las dos, creo que tiene que ver con alguna configuración de mi pc

In [10]:
#generar_clave(300)
#encoded = enc(323123)
#decoded = dec(encoded)
#print(323123 == decoded)

True
